In [50]:
from IPython.core.display import display, HTML

html_code = """
<h1 style="color: #2E86C1; text-align: center; font-family: Arial, sans-serif;
           font-weight: bold; padding: 10px; border: 2px solid #2E86C1;
           border-radius: 10px; background-color: #EAF2F8;">
    Assignment for Data QA & QC Internship @ Datahut
</h1>
"""

display(HTML(html_code))


In [24]:
import pandas as pd
import numpy as np


# <div style="background-color:#E6F7FF; padding:5px; border-radius:3px;">
# <b>📂 Load the Dataset</b>
# </div>

In [25]:
from google.colab import files
uploaded = files.upload()

Saving messy_data.csv to messy_data (1).csv


In [26]:
import pandas as pd
df = pd.read_csv("messy_data.csv")  # Use the exact uploaded file name
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11000 non-null  int64  
 1   ID          11000 non-null  object 
 2   Name        8667 non-null   object 
 3   Age         9253 non-null   float64
 4   Email       9731 non-null   object 
 5   Join Date   8808 non-null   object 
 6   Salary      8761 non-null   float64
 7   Department  8745 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 687.6+ KB


,Unnamed: 0,ID,Name,Age,Email,Join Date,Salary,Department
0,0,1e407ff9-6255-489d-a0de-34135d4f74bd,Hunter Thomas,25.0,xlopez@hotmail.com,NaN,88552.0,Sales
1,1,379f55b8-87d5-4739-a146-7400b78c24d1,Jeremy Irwin,90.0,Jillian Jenkins,2022-07-07,139227.0,NaN
2,2,18261368-dfa1-47f0-afc6-bddf45926b07,Jennifer Hammondquickly,66.0,jscottgreen.biz,2023-11-21,65550.0,Engineering
3,3,ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5,Sydney Taylorso,39.0,luke56gonzalez.com,2021-11-05,139932.0,SupportJ
4,4,14ed3e6a-e0f5-4bbe-8d93-8665267f5c90,Julia Lee,71.0,figueroakayla@yahoo.com,NaN,143456.0,Marketing


<b>🔍 Inspect the Data</b>


In [49]:
#Check for missing values
missing_values = df.isnull().sum()

# Check for duplicates
duplicates = df.duplicated().sum()

# Check column data types
data_types = df.dtypes

# Print summary
print("Missing Values:\n", missing_values)
print("\nDuplicates:", duplicates)
print("\nData Types:\n", data_types)

Missing Values:
 Unnamed: 0    0
Name          0
Age           0
Email         0
Join Date     0
Salary        0
Department    0
dtype: int64

Duplicates: 0

Data Types:
 Unnamed: 0             int64
Name                  object
Age                    int64
Email                 object
Join Date     datetime64[ns]
Salary                 int64
Department            object
dtype: object


<b>🔄 Remove Duplicates</b>

In [28]:
df=df.drop_duplicates()

In [29]:
df.duplicated().sum()

0

<b> Handle Missing Data  ❌➡️✅</b>

In [30]:
# Drop the redundant ID column if not needed
df.drop(columns=["ID"], inplace=True)


In [31]:
# Fill missing Names with 'Unknown'
df['Name'].fillna('Unknown', inplace=True)

# Remove unnecessary words/noise from names (if any)
df['Name'] = df['Name'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()

df['Name'].isnull().sum()

<ipython-input-31-264c1947ccc9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Name'].fillna('Unknown', inplace=True)


0

In [32]:
# Replace unrealistic ages (e.g., Age > 100 or < 18) with median
df['Age'] = df['Age'].apply(lambda x: np.nan if x < 18 or x > 100 else x)

# Fill missing Age with median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Convert Age to integer type
df['Age'] = df['Age'].astype(int)

df['Age'].isnull().sum()

<ipython-input-32-93b5d7aa573c>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)


0

<b>Fix Email Formats 📧 </b>

In [33]:
import re

In [34]:
# Email validation function
def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return bool(re.match(pattern, str(email)))

# Replace invalid emails with NaN
df['Email'] = df['Email'].apply(lambda x: x if validate_email(x) else np.nan)

# Drop rows where Email is missing
df.dropna(subset=['Email'], inplace=True)


df['Email'] = df['Email'].str.lower()



df.dropna(subset=['Email'], inplace=True)


In [35]:
df.Email.isnull().sum()

0

In [36]:

df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce')


df['Join Date'].fillna(df['Join Date'].median(), inplace=True)



<ipython-input-36-5257ed8097f4>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce')
<ipython-input-36-5257ed8097f4>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Join Date'].fillna(df['Join Date'].median(), inplace=True)


In [37]:
# Fill missing Salary with median salary
df['Salary'].fillna(df['Salary'].median(), inplace=True)
df['Salary'].isnull().sum()

<ipython-input-37-2f4a2f1b6437>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].fillna(df['Salary'].median(), inplace=True)


0

<b>🏢 Correct Department Names</b>

In [38]:
# Fill missing Department with 'Unknown'
df['Department'].fillna('Unknown', inplace=True)
df['Department'].isnull().sum()

<ipython-input-38-1024ad047986>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Department'].fillna('Unknown', inplace=True)


0

In [39]:

df['Salary'] = df['Salary'].apply(lambda x: np.nan if x < 10000 or x > 1000000 else x)


df['Salary'].fillna(df['Salary'].median(), inplace=True)

# Convert to integer
df['Salary'] = df['Salary'].astype(int)


<ipython-input-39-319c8d36c4ad>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].fillna(df['Salary'].median(), inplace=True)


In [41]:
df['Department'].unique()

array(['Sales', 'Marketing', 'Support', 'HR', 'Engineering', 'Unknown'],
      dtype=object)

In [42]:
# Standard department names
valid_departments = ['HR', 'Engineering', 'Marketing', 'Sales', 'Support']


def clean_department(dept):
    if isinstance(dept, str):
        for valid_dept in valid_departments:
            if valid_dept.lower() in dept.lower():
                return valid_dept
    return 'Unknown'

# Apply function to standardize department names
df['Department'] = df['Department'].apply(clean_department)


In [43]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)


In [44]:
# Check missing values after cleaning
print("\nMissing Values After Cleaning:\n", df.isnull().sum())

df.head()



Missing Values After Cleaning:
 Unnamed: 0    0
Name          0
Age           0
Email         0
Join Date     0
Salary        0
Department    0
dtype: int64


,Unnamed: 0,Name,Age,Email,Join Date,Salary,Department
0,0,Hunter Thomas,25,xlopez@hotmail.com,1997-03-10,88552,Sales
4,4,Julia Lee,71,figueroakayla@yahoo.com,1997-03-10,143456,Marketing
5,5,Lisa Bailey,81,milleryvonne@fowler-york.com,2016-10-15,123018,Support
8,8,Unknown,71,andrew74@hotmail.com,1997-03-10,83354,Sales
9,9,Jay Fisher,44,denisemaynard@jordan.com,1997-03-10,37701,HR


In [45]:
# Save cleaned dataset
df.to_csv("cleaned_dataset.csv", index=False)

print("✅ Data cleaning completed! Cleaned dataset saved as 'cleaned_dataset.csv'.")


✅ Data cleaning completed! Cleaned dataset saved as 'cleaned_dataset.csv'.


In [51]:
from google.colab import files

# Download the cleaned dataset
files.download("cleaned_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>